In [167]:

import pandas as pd
import seaborn as sb
import numpy as np
import math as m
import os
import csv
import random as rn
from matplotlib import pyplot as plt
import scipy as sp
from scipy.stats import truncnorm as tn

# Quantities to seed

papers_number = 5000
readers_number = 2500
authors_number = 25

papers = np.arange(papers_number)
readers = np.arange(readers_number)
authors = np.arange(authors_number)

# Seed folder path

dataset_name = "seed_2/p_1"
dataset_folder_path = f"../data/{dataset_name}/"
info_file_path = f"{dataset_folder_path}info.csv"
ratings_file_path = f"{dataset_folder_path}ratings.csv"
authors_file_path = f"{dataset_folder_path}authors.csv"

os.makedirs(dataset_folder_path, exist_ok=True)

print("DATASET NAME: ", dataset_name)
print("DATASET FOLDER PATH: ", dataset_folder_path)
print("INFO FILE PATH: ", info_file_path)
print("RATINGS FILE PATH: ", ratings_file_path)
print("AUTHORS FILE PATH: ", authors_file_path)


DATASET NAME:  seed_2/p_1
DATASET FOLDER PATH:  ../data/seed_2/p_1/
INFO FILE PATH:  ../data/seed_2/p_1/info.csv
RATINGS FILE PATH:  ../data/seed_2/p_1/ratings.csv
AUTHORS FILE PATH:  ../data/seed_2/p_1/authors.csv


In [168]:

# Papers distribution generation

print("---------- PAPER DISTRIBUTIONS GENERATION STARTED ----------")

paper_distributions = np.empty(papers_number)
for index in range(0, papers_number):
    percentage = 100*index/papers_number
    if percentage % 10 == 0:
        print(f"{int(index)}/{papers_number} ({int(percentage)}/100%)")
    distribution = tn(0, 1, loc=rn.uniform(0, 1), scale=rn.uniform(0, 0.05)).rvs(1)
    paper_distributions[index] = distribution
print(f"{papers_number}/{papers_number} (100/100%)")
    
print("---------- PAPER DISTRIBUTIONS GENERATION COMPLETED ----------")


---------- PAPER DISTRIBUTIONS GENERATION STARTED ----------
0/5000 (0/100%)
500/5000 (10/100%)
1000/5000 (20/100%)
1500/5000 (30/100%)
2000/5000 (40/100%)
2500/5000 (50/100%)
3000/5000 (60/100%)
3500/5000 (70/100%)
4000/5000 (80/100%)
4500/5000 (90/100%)
5000/5000 (100/100%)
---------- PAPER DISTRIBUTIONS GENERATION COMPLETED ----------


In [169]:

# Ratings file generation

# N sets of readers, each one has X% of the total

readers_percent = 20
reader_sets_number = m.floor(100 / readers_percent)
readers_amount = m.floor((readers_number*readers_percent)/100)

readers_set = set(readers)
readers_sets = []

# Readers rate papers with a certain frequence

paper_frequencies = [2, 4, 8, 30, 90]

print("---------- READERS SETS GENERATION STARTED ----------")

ratings_number = sum(paper_frequencies) * readers_amount
for x in range(0, reader_sets_number):
    current_readers_set = rn.sample(readers_set, readers_amount)
    readers_sets.append(current_readers_set)
    for reader in current_readers_set:
        readers_set.remove(reader)
    print(f"SET {x}: ", current_readers_set)
     
print("---------- READERS SETS GENERATION COMPLETED ----------")

print("---------- RATINGS GENERATION STARTED ----------")

generated_ratings = 0
with open(ratings_file_path, mode='w', newline='') as ratings_file:
    ratings_writer = csv.writer(ratings_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    ratings_writer.writerow(['Timestamp', 'Reader', 'Paper', 'Score'])
    for current_set in range(0, reader_sets_number):
        paper_per_reader = paper_frequencies[current_set]
        readers_set = readers_sets[current_set]
        for reader in readers_set:
            for index, paper in enumerate(rn.sample(set(papers), paper_per_reader)):
                paper_distribution = paper_distributions[paper]
                percentage = 100*generated_ratings/ratings_number
                if percentage % 10 == 0:
                    print(f"{int(generated_ratings)}/{ratings_number} ({int(percentage)}/100%)")
                current_tuple = {
                    "Reader": reader, 
                    "Paper": paper, 
                    "Score": round(paper_distribution, 2), 
                }
                ratings_writer.writerow([generated_ratings, current_tuple["Reader"], current_tuple["Paper"], current_tuple["Score"]])
                generated_ratings+=1
    print(f"{ratings_number}/{ratings_number} (100/100%)")
ratings_file.close()

print("---------- RATINGS GENERATION ENDED ----------")


---------- READERS SETS GENERATION STARTED ----------
SET 0:  [720, 131, 276, 1849, 1355, 196, 894, 1268, 456, 903, 1561, 810, 1424, 2154, 2046, 457, 1858, 2191, 146, 424, 1131, 498, 1117, 1331, 2353, 1719, 304, 2007, 811, 1027, 1387, 1738, 956, 1132, 1384, 1864, 1262, 1780, 2333, 2476, 2317, 1773, 663, 1908, 2039, 1937, 223, 1168, 781, 690, 1877, 579, 722, 1630, 1012, 2314, 2077, 2143, 1396, 271, 610, 60, 1627, 1667, 1633, 2339, 61, 178, 63, 577, 1148, 2489, 2181, 259, 1130, 466, 1414, 1690, 2364, 1713, 1018, 279, 2466, 1229, 618, 1934, 2392, 330, 2050, 708, 773, 1452, 2372, 1625, 1371, 2497, 1575, 161, 1959, 1758, 2458, 137, 668, 989, 974, 1793, 197, 1100, 1476, 43, 796, 1137, 124, 755, 2193, 1919, 408, 606, 1489, 2307, 1800, 2234, 1809, 2253, 136, 492, 407, 860, 673, 1933, 309, 942, 602, 435, 2388, 219, 552, 2470, 2474, 1642, 823, 392, 340, 597, 743, 566, 1837, 1180, 2049, 1286, 1110, 809, 1975, 1392, 1882, 738, 2422, 1075, 1089, 791, 1402, 139, 996, 688, 282, 1609, 153, 1090, 2176,

In [170]:

# Authors file generation

print("---------- AUTHORS GENERATION STARTED ----------")

with open(authors_file_path, mode='w', newline='') as authors_file:
    authors_writer = csv.writer(authors_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    authors_writer.writerow(["Author", "Paper"])
    for index, author in enumerate(authors):
        percentage = 100*index/authors_number
        if percentage % 10 == 0:
            print(f"{int(index)}/{authors_number} ({int(percentage)}/100%)")
        # An author writes a number of paper between 1 and paper_fraction
        author_papers_number = rn.randint(1, (papers_number-1))
        papers_written = np.random.choice(papers, author_papers_number).tolist()
        papers_written = set(papers_written)
        if len(papers_written) > 1:
            papers_written = map(str, list(papers_written))
            papers_written = ";".join(papers_written)
        authors_writer.writerow([author, papers_written])
    print(f"{authors_number}/{authors_number} (100/100%)")
authors_file.close()
        
print("---------- AUTHORS GENERATION ENDED ----------")


---------- AUTHORS GENERATION STARTED ----------
0/25 (0/100%)
5/25 (20/100%)
10/25 (40/100%)
15/25 (60/100%)
20/25 (80/100%)
25/25 (100/100%)
---------- AUTHORS GENERATION ENDED ----------


In [171]:

# Info file generation

print("---------- INFO GENERATION STARTED ----------")

info_dataframe = pd.DataFrame(columns=["Dataset", "Paper", "Reader", "Rating", "Author"])
info_dataframe = info_dataframe.append(
    {
        "Dataset": dataset_name, 
        "Paper": papers_number, 
        "Reader": readers_number, 
        "Rating": ratings_number, 
        "Author": authors_number
    }, ignore_index=True)
info_dataframe.to_csv(info_file_path, index=False)

print("---------- INFO GENERATION ENDED ----------")

---------- INFO GENERATION STARTED ----------
---------- INFO GENERATION ENDED ----------
